In [ ]:
import numpy as np
import tensorflow as tf
from keras.saving import load_model

# --- Load the serializable model (PixelShuffle version) ---
model = load_model(
    "/Users/kunalnarwani/Desktop/Thesis/unet-thesis/Super-resolution-new/models/best_by_psnr_clean.keras",
    compile=False
)

# ---------- Build a NON-repeating eval dataset ----------
# Reuse your helpers already defined: load_image, preprocess_valid, valid_hr_paths, BATCH_SIZE
eval_ds = (
    tf.data.Dataset
      .from_tensor_slices(valid_hr_paths)
      .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
      .map(preprocess_valid, num_parallel_calls=tf.data.AUTOTUNE)  # yields (lr, hr) in [0,1]
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

# ---------- Batched evaluation ----------
all_psnr = []
all_ssim = []
all_msssim = []
n_images = 0

for lr_b, hr_b in eval_ds:
    # Forward pass (faster than model.predict for small loops)
    pred_b = model(lr_b, training=False)

    # If model output size differs from HR, resize the whole batch once
    if pred_b.shape[1:3] != hr_b.shape[1:3]:
        pred_b = tf.image.resize(pred_b, size=hr_b.shape[1:3], method="bicubic")

    # Ensure valid range/dtypes for metrics
    hr_tf   = tf.cast(hr_b, tf.float32)
    pred_tf = tf.cast(tf.clip_by_value(pred_b, 0.0, 1.0), tf.float32)

    # Batched metrics (each returns shape [batch,])
    psnr_b   = tf.image.psnr(hr_tf, pred_tf, max_val=1.0)
    ssim_b   = tf.image.ssim(hr_tf, pred_tf, max_val=1.0)
    mssim_b  = tf.image.ssim_multiscale(hr_tf, pred_tf, max_val=1.0)

    # Collect as numpy
    all_psnr.append(psnr_b.numpy())
    all_ssim.append(ssim_b.numpy())
    all_msssim.append(mssim_b.numpy())

    n_images += int(hr_b.shape[0])

# ---------- Aggregate ----------
psnr_vals   = np.concatenate(all_psnr,   axis=0)
ssim_vals   = np.concatenate(all_ssim,   axis=0)
msssim_vals = np.concatenate(all_msssim, axis=0)

def mean_std(x):
    x = np.asarray(x, dtype=np.float64)
    return float(np.mean(x)), float(np.std(x))

m_psnr, s_psnr     = mean_std(psnr_vals)
m_ssim, s_ssim     = mean_std(ssim_vals)
m_msssim, s_msssim = mean_std(msssim_vals)

print(f"Validation images evaluated: {n_images}")
print(f" PSNR    : {m_psnr:.4f} ± {s_psnr:.4f} dB")
print(f" SSIM    : {m_ssim:.4f} ± {s_ssim:.4f}")
print(f" MS-SSIM : {m_msssim:.4f} ± {s_msssim:.4f}")

In [ ]:
# === OPTIONAL: VISUAL CHECK (enhanced) ===
import numpy as np
import matplotlib.pyplot as plt
import cv2

for lr, hr in valid_ds.take(1):
    pred = model.predict(lr, verbose=1)
    i = 1

    # Tensors -> np and basic prep
    hr_i   = hr[i].numpy()                        # (H, W, 3), float32 in [0,1]
    lr_i   = lr[i].numpy()                        # (h, w, 3), float32 in [0,1]
    pred_i = pred[i]                              # (H?, W?, 3), float32 (0..1-ish)

    # If output size differs from HR, resize prediction (and LR for fair side-by-side)
    H, W = hr_i.shape[:2]
    if pred_i.shape[:2] != (H, W):
        pred_i = cv2.resize(pred_i, (W, H), interpolation=cv2.INTER_CUBIC)
    if lr_i.shape[:2] != (H, W):
        lr_up = cv2.resize(lr_i, (W, H), interpolation=cv2.INTER_CUBIC)
    else:
        lr_up = lr_i

    # Clip to [0,1] for safe display
    hr_i   = np.clip(hr_i,   0, 1)
    lr_up  = np.clip(lr_up,  0, 1)
    pred_i = np.clip(pred_i, 0, 1)

    # --- 1) Absolute difference heatmap (HR vs Pred) ---
    diff = np.abs(hr_i - pred_i)                 # (H, W, 3)
    diff_gray = diff.mean(axis=2)                # (H, W)

    # --- 2) Auto-zoom around the largest error ---
    def crop_around(yx, size, H, W):
        cy, cx = yx
        hh = size // 2
        y1 = max(0, cy - hh); y2 = min(H, cy + hh)
        x1 = max(0, cx - hh); x2 = min(W, cx + hh)
        return slice(y1, y2), slice(x1, x2)

    maxy, maxx = np.unravel_index(np.argmax(diff_gray), diff_gray.shape)
    crop_size = min(128, min(H, W))  # reasonable default
    ys, xs = crop_around((maxy, maxx), crop_size, H, W)

    hr_crop   = hr_i[ys, xs]
    pred_crop = pred_i[ys, xs]
    diff_crop = diff_gray[ys, xs]

    # --- 2a) Corresponding LR crops (native and upsampled) ---
    # Map HR crop window to LR-native coordinates
    h_lr, w_lr = lr_i.shape[:2]
    sy = h_lr / float(H)
    sx = w_lr / float(W)

    lr_y1 = int(round(ys.start * sy)); lr_y2 = int(round(ys.stop * sy))
    lr_x1 = int(round(xs.start * sx)); lr_x2 = int(round(xs.stop * sx))

    # Keep indices in bounds and ensure at least 1 pixel
    lr_y1 = np.clip(lr_y1, 0, max(0, h_lr - 1))
    lr_y2 = np.clip(max(lr_y2, lr_y1 + 1), 1, h_lr)
    lr_x1 = np.clip(lr_x1, 0, max(0, w_lr - 1))
    lr_x2 = np.clip(max(lr_x2, lr_x1 + 1), 1, w_lr)

    lr_native_crop = lr_i[lr_y1:lr_y2, lr_x1:lr_x2]                   # native LR crop
    # Upsample native LR crop to the same display size as HR crop (to see blockiness)
    lr_native_crop_up = cv2.resize(
        lr_native_crop,
        (hr_crop.shape[1], hr_crop.shape[0]),
        interpolation=cv2.INTER_NEAREST
    )
    # Also crop from the already upsampled LR image (same window as HR)
    lr_up_crop = lr_up[ys, xs]

    # --- 3) Edge maps (Sobel) + edge difference (full image) ---
    def to_gray_u8(x):
        g = (0.299*x[...,0] + 0.587*x[...,1] + 0.114*x[...,2])
        g = np.clip(g*255.0, 0, 255).astype(np.uint8)
        return g

    hr_g   = to_gray_u8(hr_i)
    pred_g = to_gray_u8(pred_i)

    sobelx_hr = cv2.Sobel(hr_g,   cv2.CV_32F, 1, 0, ksize=3)
    sobely_hr = cv2.Sobel(hr_g,   cv2.CV_32F, 0, 1, ksize=3)
    edges_hr  = np.hypot(sobelx_hr, sobely_hr)

    sobelx_pr = cv2.Sobel(pred_g, cv2.CV_32F, 1, 0, ksize=3)
    sobely_pr = cv2.Sobel(pred_g, cv2.CV_32F, 0, 1, ksize=3)
    edges_pr  = np.hypot(sobelx_pr, sobely_pr)

    edge_diff = np.abs(edges_hr - edges_pr)

    def norm01(x):
        x = x.astype(np.float32)
        m, M = np.min(x), np.max(x)
        return (x - m) / (M - m + 1e-8)

    edges_hr_n  = norm01(edges_hr)
    edges_pr_n  = norm01(edges_pr)
    edge_diff_n = norm01(edge_diff)

    # --- Plot grid ---
    # Row1: HR / LR-up / Pred / Abs Diff / Edge Diff (full image)
    # Row2: HR (zoom) / LR (native→upsampled, zoom) / LR-up (zoom) / Pred (zoom) / Diff (zoom)
    fig, axs = plt.subplots(2, 5, figsize=(20, 8))

    axs[0,0].imshow(hr_i);     axs[0,0].set_title("HR");                 axs[0,0].axis("off")
    axs[0,1].imshow(lr_up);    axs[0,1].set_title("LR (upsampled)");     axs[0,1].axis("off")
    axs[0,2].imshow(pred_i);   axs[0,2].set_title("Pred");               axs[0,2].axis("off")
    im = axs[0,3].imshow(diff_gray, cmap="hot")
    axs[0,3].set_title("Abs Diff (HR − Pred)"); axs[0,3].axis("off")
    axs[0,4].imshow(edge_diff_n, cmap="gray")
    axs[0,4].set_title("Edge Difference (Sobel)"); axs[0,4].axis("off")

    axs[1,0].imshow(hr_crop);             axs[1,0].set_title("HR (zoom)");                    axs[1,0].axis("off")
    axs[1,1].imshow(lr_native_crop_up);   axs[1,1].set_title("LR (native→upsampled, zoom)");  axs[1,1].axis("off")
    axs[1,2].imshow(lr_up_crop);          axs[1,2].set_title("LR-up (zoom)");                 axs[1,2].axis("off")
    axs[1,3].imshow(pred_crop);           axs[1,3].set_title("Pred (zoom)");                  axs[1,3].axis("off")
    im2 = axs[1,4].imshow(diff_crop, cmap="hot")
    axs[1,4].set_title("Diff (zoom)");    axs[1,4].axis("off")

    # Colorbars for the two heatmaps
    cbar1 = fig.colorbar(im,  ax=axs[0,3], fraction=0.046, pad=0.04)
    cbar2 = fig.colorbar(im2, ax=axs[1,4], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()
    break